# The Battle of Neighborhoods
Coursera's "Applied Data Science Capstone", Week 4 and 5 Assignment

## Section 1. Background and Problem

### I love this city, BUT

Let me begin by introducing myself a little bit. 
We moved from another country 5 years ago and started living in THE "Silicon Valley", California, U.S.A.
We love everything about the area, especially the cultural diversity!
Everyone is so different and we love the fact that being different is okay, actually it's what makes everyone so unique. 

BUT, the real estate here is a disaster. Who knew this is place has the highest median rent in the country??
If we could live in a similar city in the States with similar multi-cultural environment, where would that be?
Is it New York? Is it Portland? Is it Seattle? Or even Austin? 
How are they similar or different from San Francisco?

### Target Audience

The target audience is someone like us, an immigrant who has little or no knowledge about the differences or similarities between the major cities in the United States. 
By grouping the similar cities together by K-means clustering, we can have some kind of idea where we want to move to if we wanted to.

In addition, the clustered result may benefit real estate companies to invest in cities where it shares the similar characteristic to City A and suggest the location to people who wants to relocate like us.


## Section 2. Data, and how it be used to solve the problem

I will be mainly using the below three data to solve the problem.


### 1) List of United States cities by population

First I would need a list of major cities in the States and if the data has some additional information about the city, that would be great. 
I came across this convenient table in the wikipedia!

https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population

Let me show you the example of the data and while I'm at it, I'm going to scrape the table. 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

website_text = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'}) 
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['2018 Rank', 'City', 'State', '2018 estimate', '2010 Census', 'Change', '2016 land area mile','2016 land area km', '2016 population density mi','2016 population density km', 'location'])
df.drop(df.index[0], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,2018 Rank,City,State,2018 estimate,2010 Census,Change,2016 land area mile,2016 land area km,2016 population density mi,2016 population density km,location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


### 2) Foursquare location data

Secondly, I need location data to compare each city. That's were the Foursquare location data comes in. 
I know from the labs and the assignments from week 3 that Foursquare location data returns nearby venues when given a coordinate. 

The data from above 1. List of United States cities by population also has the city coordinates in the table so I'll be able to utililze that information as well.
Hopefully I'll be able to generate a map with a k-means cluster like we did in the lab and also in the week 3's assignment.

### 3) Latest rent data for city and state

I am still debating whether to actively use the latest rent data in the analysis or not. 
The reason is, do I really want to live somewhere that has a similar rent compared to the current location?
Wouldn't it make sence to find somewhere that has a similar location characteristic obtained by the Foursquare?
That being considered, there is a high possibility that I'll just have the latest rent data nearby, and use that as a reference to sort the list of similar cities in the end. 

Luckily, I found a site that provides the lates rent data for city and state here: 

https://www.apartmentlist.com/rentonomics/rental-price-data/

I can also download a csv, so I did to a dataframe shown below. 
As it is sorted by the States first, the head() gives the State information first and the tail() gives City, ST information. 

By the way, the city where I live in is ranked NUMBER ONE, which is not something I find funny.

In [2]:
import pandas as pd

!wget -O rent.csv https://www.apartmentlist.com/rentonomics/wp-content/uploads/2019/12/Apartment-List-Rent-Data-Summary_2019-11.csv
rent = pd.read_csv("rent.csv")
rent.head()

--2019-12-18 22:38:10--  https://www.apartmentlist.com/rentonomics/wp-content/uploads/2019/12/Apartment-List-Rent-Data-Summary_2019-11.csv
Resolving www.apartmentlist.com (www.apartmentlist.com)... 151.101.50.217
Connecting to www.apartmentlist.com (www.apartmentlist.com)|151.101.50.217|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43844 (43K) [text/csv]
Saving to: ‘rent.csv’

100%[======================================>] 43,844      --.-K/s   in 0.003s  

2019-12-18 22:38:10 (12.1 MB/s) - ‘rent.csv’ saved [43844/43844]



,Location,Location_Type,Year,Month,Price_Studio,Price_1br,Price_2br,Price_3br,Price_4br,Price_Change_MoM,Price_Change_YoY
0,National,National,2019,11,839,960,1191,1600,1865,0.0,1.4
1,Alabama,State,2019,11,623,678,824,1101,1280,0.4,1.9
2,Alaska,State,2019,11,812,928,1187,1677,2014,0.1,-0.3
3,Arizona,State,2019,11,734,867,1095,1583,1837,0.2,3.5
4,Arkansas,State,2019,11,529,576,728,987,1156,0.0,0.9


In [3]:
rent.tail()

,Location,Location_Type,Year,Month,Price_Studio,Price_1br,Price_2br,Price_3br,Price_4br,Price_Change_MoM,Price_Change_YoY
708,"Yakima, WA",City,2019,11,604,705,914,1261,1448,-0.2,0.8
709,"Youngstown, OH",City,2019,11,515,577,727,945,1051,-0.2,-0.9
710,"Ypsilanti, MI",City,2019,11,815,868,1047,1438,1804,0.0,2.6
711,"Yucca Valley, CA",City,2019,11,578,692,865,1216,1497,0.2,3.0
712,"Zion, IL",City,2019,11,776,900,1059,1354,1611,-0.4,1.3
